# Test for Models module

## Librarie

In [4]:
import os, sys

sys.path.insert(0, os.pardir)

import torch
from torch import nn
import numpy as np

from src.models.unet3d import UNet3d

## Model

### Try Half precision

In [2]:
try:
    c = nn.Conv3d(1, 32, 3).half()
    inputs = torch.randn(8, 1, 8, 256, 256).half()
    outputs = c(inputs)
    display(inputs.shape)
    display(outputs.shape)
except Exception as e:
    print(e)

"compute_columns3d" not implemented for 'Half'


### Try on GPU

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
try:
    c = nn.Conv3d(1, 32, 3).to(device=device)
    inputs = torch.randn(8, 1, 8, 256, 256).to(device=device)
    outputs = c(inputs)
    display(inputs.shape)
    display(outputs.shape)
except Exception as e:
    print(e)

Conv3D is not supported on MPS


### From 3D Images To 2D Mask

#### Max Pooling 3D

In [4]:
# pool of non-square window
m = nn.AdaptiveMaxPool3d((1, 256, 256))
inputs = torch.randn(8, 1, 64, 256, 256)
outputs = m(inputs)

display(inputs.shape)
display(outputs.shape)

torch.Size([8, 1, 64, 256, 256])

torch.Size([8, 1, 1, 256, 256])

#### Linear 

In [5]:
m = nn.Linear(20, 30)
input = torch.randn(128, 20)
output = m(input)
display(input.shape)
display(output.size())

torch.Size([128, 20])

torch.Size([128, 30])

In [6]:
input = torch.randn(32, 1, 5, 5)
display(input.shape)
# With default parameters
m = nn.Flatten()
output = m(input)
display(output.size())
# With non-default parameters
m = nn.Flatten(0, 2)
output = m(input)
display(output.size())

torch.Size([32, 1, 5, 5])

torch.Size([32, 25])

torch.Size([160, 5])

In [5]:
inputs = torch.randn(8, 1, 8, 256, 256)

### UNet3D

In [6]:
list_channels = [1, 32, 64]
unet3d = UNet3d(list_channels)
unet3d(inputs)

tensor([[[-0.2429, -0.1109, -0.1796,  ..., -0.0175,  0.2780, -0.1272],
         [-0.1627, -0.1090, -0.2130,  ...,  0.0763,  0.0143,  0.2479],
         [-0.2614, -0.1307,  0.2664,  ...,  0.0660,  0.2463, -0.0158],
         ...,
         [-0.2740,  0.1821,  0.1675,  ..., -0.0849,  0.3529, -0.0006],
         [-0.2410,  0.1646,  0.1002,  ...,  0.1252,  0.0027, -0.0861],
         [-0.1596, -0.1959,  0.0088,  ..., -0.0131, -0.2015, -0.1626]],

        [[-0.3168,  0.0542, -0.2880,  ..., -0.0314,  0.2389, -0.2243],
         [-0.2966,  0.1489, -0.0021,  ..., -0.0836,  0.3102,  0.0105],
         [-0.3180,  0.0012,  0.0494,  ...,  0.0106,  0.1391, -0.1041],
         ...,
         [-0.1492, -0.2013,  0.1654,  ...,  0.1220, -0.1683,  0.0610],
         [-0.2024, -0.0510, -0.0352,  ...,  0.3232,  0.0719, -0.1029],
         [-0.2086, -0.1803, -0.1657,  ..., -0.2263, -0.1479, -0.3583]],

        [[-0.4633, -0.3944, -0.1506,  ...,  0.1481,  0.3138, -0.2984],
         [-0.2961, -0.2286, -0.3898,  ...,  0

# Pytorch-Lightning

## Metrics

In [19]:
from torchmetrics.classification import BinaryFBetaScore

threshold = .5
f05score = BinaryFBetaScore(0.5, threshold)
targets = torch.randint(0, 2, (8000,1))
predictions = torch.randint(0, 2, (8000,1))
f05score(targets, targets)

tensor(1.)

In [1]:
import os, sys

sys.path.insert(0, os.pardir)

from src.data.make_dataset import CustomDataset
from src.models.metrics import F05Score
from constant import TRAIN_FRAGMENTS
from torch.utils.data import DataLoader
from src.utils import get_device
from src.data.make_dataset import get_mask_sizes

dataset = CustomDataset(['1'], augmentation=False)
dataloader = DataLoader(dataset=dataset, batch_size=128)
mask_sizes = get_mask_sizes(['1'])
device = get_device()
metric = F05Score(mask_sizes, 0.5).to(device=device)
metric_noise = F05Score(mask_sizes, 0.5).to(device=device)

In [2]:
import torch
import numpy as np

for i, (indexes, inputs, masks, coords) in enumerate(dataloader):
    # This metric have to give a score of 1
    metric.update(masks, masks, coords, indexes)
    outputs = torch.tensor(np.zeros(masks.shape, np.int16)).to(device=device)
    # This metric have to give a score of ~0.5
    metric_noise.update(outputs, masks, coords, indexes)
    # if i > 10:
    # break

print('Perfect F05Score:', metric.compute())
print('Noisy F05Score:', metric_noise.compute())
metric.reset()
metric_noise.reset()

Perfect F05Score: (tensor(1., device='mps:0'), tensor(1., device='mps:0'))
Noisy F05Score: (tensor(0., device='mps:0'), tensor(0., device='mps:0'))


## Losses

In [3]:
import os, sys

sys.path.insert(0, os.pardir)

from src.data.make_dataset import CustomDataset
from torch.utils.data import DataLoader
from src.utils import get_device

device = get_device()
from src.models.losses import CombinedLoss

dataset = CustomDataset(['1'], augmentation=False)
dataloader = DataLoader(dataset=dataset, batch_size=128)
criterion = CombinedLoss(bce_weight=1)

In [9]:
import torch
from torch.nn import BCELoss
import numpy as np

for i, (indexes, inputs, masks, coords) in enumerate(dataloader):
    x = criterion(masks, masks)
    outputs = torch.tensor(np.zeros(masks.shape, np.float32)).to(device=device)
    x = bce(outputs, masks)
    print(x)
    break

tensor(29.4795, device='mps:0')
